In [3]:
####### 데이터 준비 
from keras.layers import Embedding
from keras.datasets import imdb
from keras import preprocessing

max_features = 10000 # 특성으로 사용할 단어의 수
maxlen = 20  # 사용할 텍스트의 길이(가장 빈번한 20 개의 단어만 사용합니다)

# 정수 리스트로 데이터를 로드합니다.
import numpy as np
old = np.load
np.load = lambda *a,**k: old(*a,**k,allow_pickle=True)

from keras.datasets import imdb
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

np.load = old
del(old)

# 리스트를 (samples, maxlen) 크기의 2D 정수 텐서로 변환.
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen) # 가장 빈번한 단어 최대 maxlen(20)개만
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)
print(x_train.shape)

####### 모델 생성

from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding

model = Sequential()

# 가능한 토큰의 개수(1,0000)와 임베딩 차원(8)로 설정 
model.add(Embedding(10000, 8, input_length=maxlen))
# Embedding 층의 출력 크기는 (samples, maxlen, 8)가 됨

model.add(Flatten())  #(samples, maxlen * 8) 크기의 2D 텐서로 펼침


# 분류기를 추가
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])
model.summary()


####### 학습

history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_split=0.2)


""" 리뷰에서 20개의 단어만 사용한 것 치고는 70%이상의 좋은 결과. 
하지만,각 단어를 독립적으로 다루었음. 
단어 사이의 관계나 문장의 구조를 고려하지 않았음. 
각 시퀀스 전체를 고려한 특성을 학습하도록 임베딩 층 위에 순환 층이나 1D 합성곱 층을 추가하는 것이 좋음. 
"""

W0806 10:43:52.141455  1032 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0806 10:43:52.163373  1032 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0806 10:43:52.166368  1032 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0806 10:43:52.205262  1032 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0806 10:43:52.218227  1032 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3

(25000, 20)
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten_1 (Flatten)          (None, 160)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


W0806 10:43:52.421954  1032 deprecation_wrapper.py:119] From C:\Users\user\Anaconda3\lib\site-packages\keras\backend\tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 1s 54us/step - loss: 0.6759 - acc: 0.6049 - val_loss: 0.6398 - val_acc: 0.6814
Epoch 2/10
20000/20000 [==============================] - 1s 41us/step - loss: 0.5658 - acc: 0.7427 - val_loss: 0.5467 - val_acc: 0.7206
Epoch 3/10
20000/20000 [==============================] - 1s 41us/step - loss: 0.4752 - acc: 0.7808 - val_loss: 0.5113 - val_acc: 0.7384
Epoch 4/10
20000/20000 [==============================] - 1s 41us/step - loss: 0.4263 - acc: 0.8077 - val_loss: 0.5008 - val_acc: 0.7452
Epoch 5/10
20000/20000 [==============================] - 1s 41us/step - loss: 0.3930 - acc: 0.8258 - val_loss: 0.4981 - val_acc: 0.7536
Epoch 6/10
20000/20000 [==============================] - 1s 41us/step - loss: 0.3668 - acc: 0.8395 - val_loss: 0.5014 - val_acc: 0.7530
Epoch 7/10
20000/20000 [==============================] - 1s 41us/step - loss: 0.3435 - acc: 0.8533 - val_loss: 0.5052 - val_acc:

' 리뷰에서 20개의 단어만 사용한 것 치고는 70%이상의 좋은 결과. \n하지만,각 단어를 독립적으로 다루었음. \n단어 사이의 관계나 문장의 구조를 고려하지 않았음. \n각 시퀀스 전체를 고려한 특성을 학습하도록 임베딩 층 위에 순환 층이나 1D 합성곱 층을 추가하는 것이 좋음. \n'

In [18]:
# 임의 데이터 테스트
from nltk import word_tokenize
from keras.preprocessing import sequence
import nltk


test_review = """it wasn't impressed. it's just... The contents were not delivered accurately"""
tokens = word_tokenize(test_review)
word_index = imdb.get_word_index()

result = []
for i in tokens:
    idx = word_index.get(i)
    if idx!=None and idx<10000:
        result.append(word_index.get(i))

test_x = sequence.pad_sequences([result], maxlen=20)
model.predict(test_x) # 부정
    

array([[0.05341413]], dtype=float32)